In [14]:
%load_ext autoreload
%autoreload 2
# So code is automatically reloaded when saved in different modules

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [59]:
  
import os
import pickle
import numpy as np
import tensorflow as tf
from scipy import stats
import tensorflow as tf
from sklearn.metrics import roc_auc_score
import time
import logging
import sys
import time
from datetime import datetime
import csv
from collections import Counter, OrderedDict, defaultdict
import sys
sys.path.append('src/taggerSystem/')
from my_util import print_sentence, write_conll, read_conll
from my_data_util import load_and_preprocess_data, load_embeddings, ModelHelper
logger = logging.getLogger("hw3.q2")
logger.setLevel(logging.DEBUG)
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.DEBUG)

## File Paths


In [51]:
data_train = "data/smallIcd9NotesDataTable_test.csv"
data_valid = "data/smallIcd9NotesDataTable_valid.csv"
vocab = "src/taggerSystem/data_hw3_delete/vocab.txt"
wordVecs = "src/taggerSystem/data_hw3_delete/wordVectors.txt"
output_path = 'results/{}/{:%Y%m%d_%H%M%S}/".format(self.cell, datetime.now())'
log_output = output_path + "log"
START_TOKEN = "<s>"
END_TOKEN = "</s>"
NUM = "NNNUMMM"
UNK = "UUUNKKK"
EMBED_SIZE = 50

## Extract Data and Word Embeddings

In [88]:
helper, train, dev, train_raw, dev_raw, xTrain, yTrain, xDev, yDev = load_and_preprocess_data(
    data_train = data_train, data_valid = data_valid)
print('dev raw')
print(dev_raw)
print('dev')
print(dev)
embeddings = load_embeddings(vocab, wordVecs, helper)
helper.save(output_path)# token2id and max length saved to output_path
handler = logging.FileHandler(log_output)
handler.setLevel(logging.DEBUG)
handler.setFormatter(logging.Formatter('%(asctime)s:%(levelname)s: %(message)s'))
logging.getLogger().addHandler(handler)

INFO:Loading training data...
INFO:Done. Read 7 notes
INFO:Loading dev data...
INFO:Done. Read 7 notes
INFO:Total read time 0.008938
INFO:Built dictionary for 15 features.
INFO:There are a total of 4 ICD codes


dev raw
[(['dog', 'dog', 'dog', 'horse'], ['4240', '486']), (['cat', 'cat', 'horse'], ['7212']), (['horse', 'horse', 'horse', 'Batman'], ['4240', '486', '5845']), (['the', 'the', 'of', 'and'], ['4240']), (['the'], ['4240']), (['and'], ['4240']), (['with'], ['4240'])]
dev
[([[2, 9], [2, 9], [2, 9], [1, 9]], array([ 0.,  1.,  0.,  1.])), ([[4, 9], [4, 9], [1, 9]], array([ 0.,  0.,  1.,  0.])), ([[1, 9], [1, 9], [1, 9], [8, 10]], array([ 1.,  1.,  0.,  1.])), ([[3, 9], [3, 9], [6, 9], [5, 9]], array([ 0.,  0.,  0.,  1.])), ([[3, 9]], array([ 0.,  0.,  0.,  1.])), ([[5, 9]], array([ 0.,  0.,  0.,  1.])), ([[7, 9]], array([ 0.,  0.,  0.,  1.]))]


INFO:Initialized embeddings.


## Hyperparameters

In [98]:
tf.reset_default_graph()

learning_rate = 0.001
training_epochs = 100
batch_size = 1
total_batches = (xTrain.shape[0]//batch_size)

n_input = 1
n_steps = 10
n_hidden = 30
n_classes = helper.n_classes

alpha = 0.5

In [111]:
embeddings[0].shape

(50,)

## Placeholders

In [100]:
x = tf.placeholder("float", [None, n_steps, n_input])
y = tf.placeholder("float", [None, n_classes])
y_steps = tf.placeholder("float", [None, n_classes])

## Helper Functions

In [114]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev = 0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.0, shape = shape)
    return tf.Variable(initial)

def LSTM(x, weight, bias):
    cell = tf.contrib.rnn.BasicLSTMCell(n_hidden,state_is_tuple = True)
#     multi_layer_cell = tf.nn.rnn_cell.MultiRNNCell([cell] * 2)
    output, state = tf.nn.dynamic_rnn(cell, x, dtype = tf.float32)
    output_flattened = tf.reshape(output, [-1, n_hidden])
    output_logits = tf.add(tf.matmul(output_flattened,weight),bias)
    output_all = tf.nn.sigmoid(output_logits)
    output_reshaped = tf.reshape(output_all,[-1,n_steps,n_classes])
    output_last = tf.gather(tf.transpose(output_reshaped,[1,0,2]), n_steps - 1)  
    #output = tf.transpose(output, [1, 0, 2])
    #last = tf.gather(output, int(output.get_shape()[0]) - 1)
    #output_last = tf.nn.sigmoid(tf.matmul(last, weight) + bias)
    return output_last, output_all

In [115]:
weight = weight_variable([n_hidden,n_classes])
bias = bias_variable([n_classes])
y_last, y_all = LSTM(x,weight,bias)

## Loss Function

In [ ]:
# The paper does something weird. I'd suggest doing something closer to what we do in homework 3

## Training and Testing

In [97]:
with tf.Session() as session:
    tf.global_variables_initializer().run()
    for epoch in range(training_epochs):
        for b in range(total_batches):    
            offset = (b * batch_size) % (train_y.shape[0] - batch_size)
            batch_x = train_x[offset:(offset + batch_size), :]
            batch_y = train_y[offset:(offset + batch_size), :]
            batch_y_steps = np.tile(batch_y,((train_x.shape[1]),1))
            _, c = session.run([optimizer, loss_function],feed_dict={x: batch_x, y : batch_y, y_steps: batch_y_steps})   
        pred_y = session.run(y_last,feed_dict={x:test_x})
        print("ROC AUC Score: ",roc_auc_score(test_y,pred_y))

NameError: name 'total_batches' is not defined